## Topic: Comparative Analysis of PM2.5 Levels on Public Holidays, Weekends, and Weekdays in Seoul (by Month)

### Reason for Topic Selection
1. I was curious to see how differences in activity levels between public holidays, weekends, and weekdays influence fine dust concentrations (PM2.5), which led me to choose this topic.
2. Using data from 2023, I aimed to clearly identify monthly patterns and differences by day type through visual comparisons.
3. Initially, I considered analyzing PM2.5 data at the national level, but the differences between day types were not as distinct, which diluted the focus. As a result, I chose Seoul—a high-density, high-activity area—as the subject of this analysis.

### Analysis Summary
This project compares monthly PM2.5 levels in Seoul throughout 2023, visualizing differences across public holidays, weekends, and weekdays.


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import matplotlib
from matplotlib import font_manager

-----------------------------------------------------------------------------------------------------------------------------

In [ ]:
# 한글 폰트를 따로 설정하지 않고 시각화를 진행하였더니 폰트가 깨지는 현상일 발생하여, 
# 이를 해결하기 위해 AppleGothic 폰트를 설정합니다.
try:
    font_path = "/System/Library/Fonts/Supplemental/AppleGothic.ttf"  
    if os.path.exists(font_path):
        font_prop = font_manager.FontProperties(fname=font_path)
        matplotlib.rc('font', family=font_prop.get_name())
        sns.set(style="whitegrid", rc={"font.family": font_prop.get_name()})  
        print(f"설정된 폰트: {font_prop.get_name()}")
    else:
        print("AppleGothic 폰트를 찾을 수 없습니다. 다른 폰트를 시도하세요.")
except Exception as e:
    print(f"폰트 설정 중 오류 발생: {e}")

In [ ]:
plt.rcParams['axes.unicode_minus'] = False

공휴일 리스트를 정의하였고, 2023년에는 총 15일의 공휴일이 있습니다.
공휴일 데이터를 정의하여 데이터 전처리 및 시각화에 사용합니다.

In [ ]:
holidays = [
    "20230101", "20230121", "20230122", "20230123",
    "20230210", "20230301", "20230505", "20230606",
    "20230815", "20230928", "20230929", "20230930",
    "20231003", "20231009", "20231225"
]

(Data Loading and Preprocessing)
	1.	Loaded and combined Excel files for all 12 months of 2023 to ensure consistency across the dataset.
	2.	Created additional columns such as month, day of the week, holiday indicator, and weekend indicator using the date information.
	3.	Cleaned and organized the data to prepare it for analysis and visualization.

In [ ]:
def load_and_preprocess_data(file_paths):
    all_data = []  # 모든 데이터를 저장할 리스트를 생성합니다.(각 파일의 데이터를 저장하고 병합하기 위해 리스트 생성)
    start = time.time()
    for file_path in file_paths:
        print(f"Loading file: {file_path}")
        try:
            # 엑셀 데이터 읽어오며, 필요한 열만 로드합니다.
            data = pd.read_excel(file_path, usecols=["지역", "측정일시", "PM25"])
            data.columns = ["지역", "날짜", "PM2.5"] #데이터를 이해하기 쉽도록 컬럼 이름을 변경하였습니다.(PM25->PM2.5)
            # 데이터가 동일한 형식을 유지하기 위해 날짜 포맷을 통일합니다. 
            data["날짜"] = data["날짜"].astype(str).str.zfill(10).str[:8]
            data["월"] = pd.to_datetime(data["날짜"], format="%Y%m%d").dt.month #월 데이터를 추출합니다.
            data["요일"] = pd.to_datetime(data["날짜"], format="%Y%m%d").dt.dayofweek #dayofweek를 사용하여 0~6의 숫자로 요일 추출합니다.(0:월요일, 6:일요일)
            data["공휴일"] = data["날짜"].isin(holidays) # 공휴일 리스트에 포함된 날짜인지 여부를 boolean 값으로 추가합니다.
            data["주말"] = data["요일"].isin([5, 6]) # 요일 값이 5, 6 각각 토요일과 일요일인지 확인해 boolean 값으로 추가합니다.
            
            # 결측값 확인 및 처리합니다.
            missing_values = data.isnull().sum() #isnull().sum()을 사용하여 각 열의 결측값 개수를 계산합니다.
            print(f"결측값 확인 결과: {missing_values}") # 만약 결측값이 존재하면, dropna()로 제거합니다.(결측값으로 인해 발생할 수 있는 오류 방지)
            if missing_values.sum() > 0:
                data = data.dropna()  # 결측값 제거합니다.
            
            # 이상치 확인 및 처리합니다.
            outlier_count = data[(data["PM2.5"] < 0) | (data["PM2.5"] > 500)].shape[0] # PM2.5값이 0미만이거나 500 초가인 데이터는 이상치로 간주합니다.
            print(f"이상치 개수: {outlier_count}") # 데이터 품질을 유지하여 잘못된 결과를 방지하기 위해 PM2.5값이 유효 범위 내에 있는 데이터만 유지하도록 합니다.
            if outlier_count > 0:
                data = data[(data["PM2.5"] >= 0) & (data["PM2.5"] <= 500)]

Missing Values and Outlier Check
No significant issues were found in the dataset regarding missing values or outliers.

In [ ]:
#모든 파일의 데이터를 병합하고, 데이터를 로드하고 전처리하는데 걸리는 시간을 측정합니다.
            all_data.append(data)
        except FileNotFoundError as e:
            print(f"File not found: {file_path}")
    combined_data = pd.concat(all_data, ignore_index=True)
    print(f"데이터 로드 및 전처리 완료 (소요 시간: {time.time() - start:.2f}초)")
    return combined_data

Seoul Area Data Filtering Function
This function filters the dataset to include only records from the Seoul area.
By doing so, the analysis focuses solely on PM2.5 concentrations in Seoul.

In [ ]:
def filter_seoul_data(data):
    start = time.time()
    seoul_data = data[data["지역"].str.contains("서울")]
    print(f"서울 지역 데이터 필터링 완료 (소요 시간: {time.time() - start:.2f}초)")
    return seoul_data

(Visualization Function: Comparing Monthly PM2.5 Distributions by Holidays, Weekends, and Weekdays)
	1.	A new column labeled “category” is added to distinguish holidays, weekends, and weekdays.
	2.	A boxplot is used to visualize the distribution of PM2.5 concentrations by month and category.
	3.	This allows for an intuitive comparison of air quality differences across holidays, weekends, and weekdays.

A boxplot was chosen because it effectively displays the median, range, and outliers, making it a strong tool for identifying patterns and characteristics in monthly PM2.5 concentrations.
It also clearly reveals how the median values shift by season and highlights the differences between holidays and regular weekdays.

In [ ]:
def visualize_monthly_day_type_comparison(data):
    start = time.time()
    data = data.copy()
    data["구분"] = data.apply(
        lambda x: "공휴일" if x["공휴일"] else ("주말" if x["주말"] else "평일"), axis=1
    )
    plt.figure(figsize=(14, 8)) # 그래프 크기를 설정하였습니다.
    sns.boxplot(x="월", y="PM2.5", hue="구분", data=data, palette="Set2") #박스플롯을 생성합니다.
    plt.title("서울 지역 월별 공휴일, 주말, 평일 PM2.5 분포 비교") #그래프 제목을 설정합니다.
    plt.xlabel("월") #X축 레이블입니다.
    plt.ylabel("PM2.5 농도") #Y축 레이블입니다.
    plt.legend(title="구분") #범례 제목입니다.
    plt.grid() # 데이터를 쉽게 읽을 수 있도록 그리드를 추가합니다.
    plt.show()
    print(f"월별 공휴일, 주말, 평일 비교 시각화 완료 (소요 시간: {time.time() - start:.2f}초)")

Interpretation of the Visualization Results

1. Analysis by Day Type (Holidays, Weekends, Weekdays)

The graph shows that PM2.5 concentrations tend to be lower on holidays and weekends compared to weekdays.
This may be due to reduced vehicle traffic and industrial operations during non-working days, which leads to fewer emissions.
On specific holidays such as Lunar New Year or Chuseok, large numbers of people travel outside urban areas, possibly contributing to lower PM2.5 levels in the city center.

2. Monthly (Seasonal) Analysis

The overall PM2.5 concentration during the summer months (June to August) is relatively low, likely due to favorable atmospheric conditions.
In summer, strong air circulation and higher rainfall can help disperse fine dust particles, reducing air pollution.
In contrast, higher concentrations from January to March may be caused by increased use of heating systems and stagnant air in large cities like Seoul.
Interestingly, PM2.5 levels can be higher on weekends in early spring, possibly because people stay indoors more and use heating more intensively.
March and April also coincide with frequent yellow dust storms, which significantly increase the concentration of fine particulate matter, as reflected in the graph.


In [ ]:
# 파일 경로 리스트: 이 리스트를 활용해 반복적으로 데이터를 로드하고 처리할 수 있도록 설정합니다.
base_dir = "/Users/bisong/Desktop/빅데이터과제/2023"
file_names = [
    "2023년 1월.xlsx", "2023년 2월.xlsx", "2023년 3월.xlsx",
    "2023년 4월.xlsx", "2023년 5월.xlsx", "2023년 6월.xlsx",
    "2023년 7월.xlsx", "2023년 8월.xlsx", "2023년 9월.xlsx",
    "2023년 10월.xlsx", "2023년 11월.xlsx", "2023년 12월.xlsx"
]
file_paths = [os.path.join(base_dir, file_name) for file_name in file_names]

In [ ]:
# 실행 흐름 확인용 코드입니다.
# 각 단계별로 데이터가 올바르게 처리되고 있는지와 오류 발생시 어디서 문제가 발생하였는지를 확인하기 위해 이를 메시지로 출력할 수 있도록 하였습니다.
try:
    total_start = time.time()
#전체 작업 시간을 알기위해 코드를 작성합니다.
    print("데이터 로드 및 전처리 중이니 기다리시옵소서.")
    data = load_and_preprocess_data(file_paths)
#데이터 로드 및 전처리 과정입니다.
    print("서울 지역 데이터 필터링 중이니 기다리시옵소서.")
    seoul_data = filter_seoul_data(data)
#서울 지역 데이터 필터링하는 과정입니다.
    print("월별 공휴일, 주말, 평일 비교 시각화 중이니 기다리세요.")
    visualize_monthly_day_type_comparison(seoul_data)
#데이터를 시각화하는 과정입니다.

    print(f"전체 작업 완료 (총 소요 시간: {time.time() - total_start:.2f}초)")
except Exception as e:
    print(f"오류 발생: {e}")